# 通过Python requests爬取每周的暴发流感起数

In [7]:
# 安装requests库
!pip install requests

# 安装pandas库
!pip install pandas

# 安装xpath库
!pip install lxml -i https://pypi.tuna.tsinghua.edu.cn/simple

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'D:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'D:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


You should consider upgrading via the 'D:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
# 导入 requests和pandas 包
import requests
import pandas as pd
from lxml import etree

headers = {
    "Connection": "keep-alive",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Encoding": "gzip, deflate",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    "Cache-Control": "max-age=0",
    # "Referer": "http://www.nhc.gov.cn/xcs/yqtb/list_gzbd.shtml"
}

# 流感周报列表网址
url_string = 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/index.htm'

# 发送请求
response = requests.get(url_string, headers=headers)
print('Response code 2: ', response.status_code)

url_list = []

if response.status_code == 200:
    # 得到网页的HTML内容
    html = response.content

    #
    parser = etree.HTMLParser(encoding='utf-8')
    xpath = etree.fromstring(html, parser=parser)

    names = xpath.xpath('/html/body/div[2]/div/div[1]/div/div[2]/ul/li/span/a/text()')
    urls = xpath.xpath('/html/body/div[2]/div/div[1]/div/div[2]/ul/li/span/a/@href')
    urls = [f'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb{url[1:]}' for url in urls]

    for i in range(len(urls)):
        name = names[i]
        url = urls[i]

        print(f'name={name}, url={url}')
        url_list.append((name, url))

Response code 2:  200
urls: ['https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202304/t20230413_265257.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202304/t20230406_264879.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202303/t20230330_264734.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202303/t20230324_264512.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202303/t20230316_264314.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202303/t20230308_264131.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202303/t20230302_264010.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202302/t20230223_263876.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202302/t20230217_263778.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202302/t20230209_263680.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202302/t20230201_263574.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202301/t20230127_263524.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202301/t20230118_263470.htm', 'https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202301/

In [26]:
result_list = []

for name, url in url_list:
    print(f'Requesting: name={name}, url={url}')

    response = requests.get(url, headers=headers)
    print('Response code: ', response.status_code)

    if response.status_code == 200:
        # 返回 网页 数据
        html = response.content

        # XPath路径解析
        parser = etree.HTMLParser(encoding='utf-8')
        xpath = etree.fromstring(html, parser=parser)
        text = [i.xpath('string()') for i in xpath.xpath('/html/body/div[2]/div/div[1]/div/div[2]/div/div/div/span[1]/span/p[1]')]

        # 个别网页xpath不同
        if not text:
            text = [i.xpath('string()') for i in xpath.xpath('/html/body/div[2]/div/div[1]/div/div[2]/div/div/div/span/p[1]')]
        if not text:
            text = [i.xpath('string()') for i in xpath.xpath('/html/body/div[2]/div/div[1]/div/div[2]/div/div/div/p[5]')]

        print(f'content={text}')
        paragraph = text[0]

        try:
            index_start = paragraph.index('报告') + 2
            index_stop = paragraph.index('起', index_start)
            text = paragraph[index_start:index_stop]
        except Exception as e:
            index_start = -1
            index_stop = -1
            text = 0

        print(f'index_start={index_start}, index_stop={index_stop}, text={text}')

        result_list.append((name, text))

df = pd.DataFrame(data=result_list, index=None)
df

Requesting: name=2023 第14周, url=https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202304/t20230413_265257.htm
Response code:  200
content=['·监测数据显示，本周南方省份流感病毒检测阳性率略低于上周，北方省份持续下降。A(H1N1)pdm09（季节性甲型H1N1亚型流感病毒）为主、A(H3N2)（季节性甲型H3N2亚型流感病毒）流感病毒共同流行。南方省份流感样病例百分比开始下降，北方省份继续下降。本周共报告443起流感样病例暴发疫情。']
index_start=134, index_stop=137, text=443
Requesting: name=2023 第13周, url=https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202304/t20230406_264879.htm
Response code:  200
content=['·监测数据显示，本周南方省份流感病毒检测阳性率略低于上周，北方省份持续下降。A(H1N1)pdm09（季节性甲型H1N1亚型流感病毒）为主、A(H3N2)（季节性甲型H3N2亚型流感病毒）流感病毒共同流行。南方省份流感样病例百分比与上周接近，北方省份继续下降。本周共报告526起流感样病例暴发疫情。']
index_start=135, index_stop=138, text=526
Requesting: name=2023 第12周, url=https://ivdc.chinacdc.cn/cnic/zyzx/lgzb/202303/t20230330_264734.htm
Response code:  200
content=['·监测数据显示，本周南方省份流感病毒检测阳性率上升趋缓，北方省份持续下降。A(H1N1)pdm09（季节性甲型H1N1亚型流感病毒）为主、A(H3N2)（季节性甲型H3N2亚型流感病毒）流感病毒共同流行。南方省份流感样病例百分比略高于上周，北方省份继续下降。本周共报告683起流感样病例暴发疫情。']
index_start=134, index_stop=137, text=683
Requesting: name=2023 第11周, ur

,0,1
0,2023 第14周,443
1,2023 第13周,526
2,2023 第12周,683
3,2023 第11周,858
4,2023 第10周,807
5,2023 第9周,720
6,2023 第8周,390
7,2023 第7周,108
8,2023 第6周,9
9,2023 第5周,0


In [27]:
# 将数据保存到csv文件中
df.to_csv('C:/Users/Ruibin/Desktop/flu_weekly.csv',encoding='utf_8')